In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pydrake.all import Meshcat

from capstan_effect_test import CapstanEffectTest

In [ ]:
meshcat = Meshcat()

In [ ]:
theta_list = np.arange(0.4*np.pi, 2.01*np.pi, 0.1*np.pi)
ratio_list = []
for theta in theta_list:
    t, f1, f2 = CapstanEffectTest(theta=theta, delta_theta=0.025*np.pi, simulation_time=2, meshcat=meshcat)
    ratio = np.mean((np.abs(f2) / np.abs(f1))[-50:])
    ratio_list.append(ratio)


In [ ]:
plt.plot(theta_list, np.log(ratio_list), 'o')
plt.show()